In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
GEMINI_API_KEY=os.getenv("GEMINI_API_KEY")

In [21]:
%pip install sentence-transformers

^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached sentence_transformers-5.0.0-py3-none-any.whl (470 kB)
  Using cached transformers-4.55.0-py3-none-any.whl (11.3 MB)
  Using cached torch-2.7.1-cp311-cp311-win_amd64.whl (216.1 MB)
  Using cached scikit_learn-1.7.1-cp311-cp311-win_amd64.whl (8.9 MB)
                                              0.0/38.6 MB ? eta -:--:--
                                              0.0/38.6 MB ? eta -:--:--
                                              0.1/38.6 MB 1.2 MB/s eta 0:00:34
                                              0.2/38.6 MB 1.5 MB/s eta 0:00:26
                                              0.2/38.6 MB 1.5 MB/s eta 0:00:26
                                              0.4/38.6 MB 1.6 MB/s eta 0:00:25
                                              0.4/38.6 MB 1.6 MB/s eta 0:00:25
                                              0.4/38.6 MB 1.6 MB/s eta 0:00:25
                                              0.4/38.6 MB 1.6 MB/s eta 0:00:25
                                       

In [19]:
from langchain_huggingface import HuggingFaceEmbeddings
embaddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-V2")

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [7]:
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# embadding=GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GEMINI_API_KEY)
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")

## SIMPLAE AI ASSISTANT

In [10]:
while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

Hello! 👋  How can I help you today? 😊

As an AI language model, I don't "do" things in the way a human does. I don't have experiences or feelings.

Right now, I'm waiting for your next question or instruction. I can process information and respond to a wide range of prompts, from answering factual questions to generating creative text formats.

What can I help you with?

Hello Raghav!

It's nice to meet you. What can I do for you today? 😊  


As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to know! 😊

Goodbye! I hope you have a great day. 😊

goodbye take care yourself


In [11]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [12]:
store={}

In [13]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [14]:
config = {"configurable": {"session_id": "firstchat"}}

In [15]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)

In [16]:
model_with_memory.invoke(("Hi! I'm Raghav"),config=config).content

"Hi Raghav, it's nice to meet you!\n\nWhat can I do for you today? 😊\n"

In [17]:
model_with_memory.invoke(("tell me what is my name?"),config=config).content

'Your name is Raghav. 😊 \n\nI remember that from our first interaction!  Is there anything else I can help you with?\n'

In [18]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Raghav", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Raghav, it's nice to meet you!\n\nWhat can I do for you today? 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 16, 'total_tokens': 41, 'completion_time': 0.045454545, 'prompt_time': 0.001247759, 'queue_time': 0.25441263, 'total_time': 0.046702304}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6341ec4e-078c-42ae-be40-63a61472f086-0', usage_metadata={'input_tokens': 16, 'output_tokens': 25, 'total_tokens': 41}), HumanMessage(content='tell me what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Raghav. 😊 \n\nI remember that from our first interaction!  Is there anything else I can help you with?\n', additional_kwargs={}, response_me

## RAG

In [ ]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

### Reading the txt files from source directory

loader = DirectoryLoader('../data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
'''

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [ ]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [ ]:
question ="what is llama3? can you highlight 3 important points?"
print(retrieval_chain.invoke(question))

## Let's Start with Tools and Agents

In [23]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [27]:
api_wrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=100)

In [28]:
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [29]:
tool.name

'wikipedia'

In [30]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [31]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [32]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of 


In [33]:
from langchain_community.tools import YouTubeSearchTool

In [34]:
tool=YouTubeSearchTool()

In [35]:
tool.name

'youtube_search'

In [36]:
tool.description

'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional'

In [37]:
tool.args

{'query': {'title': 'Query', 'type': 'string'}}

In [39]:
tool.run("shivapura kampa")

"['https://www.youtube.com/watch?v=qo9wZQ91bUs&list=RDqo9wZQ91bUs&start_radio=1&pp=ygUPc2hpdmFwdXJhIGthbXBhoAcB', 'https://www.youtube.com/watch?v=V8PrsnwAUuw&pp=ygUPc2hpdmFwdXJhIGthbXBh0gcJCccJAYcqIYzv']"